In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from utils.ds_utils import load_dataset_from_huggingface
ds_train = load_dataset_from_huggingface(split="train")
ds_test = load_dataset_from_huggingface(split="test")

Loaded dataset: dragonslayer631/ci2_allsides, split: train
Loaded dataset: dragonslayer631/ci2_allsides, split: test


In [ ]:
ds_train[0]

In [43]:
from datasets import Dataset
def update_dataset_column(ds: Dataset, values: dict, column: str):
    for i in range(len(ds)):
        if ds[i]['id'] in values.keys():
            ds[i][column] = values[ds[i]['id']]
    return ds

In [55]:
from utils.groq_utils import generate_entites_sentiments_groq, get_groq_client
import json
from datasets import Dataset

client = get_groq_client()

def get_entity_sentiments_for_text_in_dataset(src: str, ds: Dataset, seen: list):
    #filtered = ds.filter(lambda x: x[f'{text}_entity_sentiments'] == None)
    print(f"Number of rows to process: {len(ds)}")
    ctr = 0
    #new_values = {}
    for elt in ds:
        ctr += 1
    
        if ctr % 1000 == 0:
            print("seen ", ctr)

        if elt['id'] in seen:
            #print("seen")
            continue
        
        print("trying ", elt["id"])
        try:
            if src == 'text':
                # snip text
                text = elt['text'].split(' ')
                text = ' '.join(text[:1000])
            else:
                text = elt[src]
            sentiments = generate_entites_sentiments_groq(client=client, text=text, model="llama3-70b-8192")
            #print("sentiments", sentiments)
            with open(f"data/{src}/{src}_sentiment_llama.csv", "a") as f:
                f.write(f"{elt['id']}|{json.dumps(sentiments)}\n")
            #new_values[elt['id']] = json.dumps(sentiments)
        except Exception as e:
            print("failed at", elt['id'], e)

    #print(f"updating {src}_entity_sentiments with values ", new_values)
    #update_dataset_column(ds, new_values, f'{text}_entity_sentiments')
    return ds
    

In [59]:
import pandas as pd

seen = pd.read_csv("data/summary_5/summary_5_sentiment_llama.csv", sep='|')["id"]

In [60]:
seen = list(set(seen))

In [61]:
get_entity_sentiments_for_text_in_dataset("summary_5", ds_train, seen)

Number of rows to process: 36071
seen  1000
seen  2000
seen  3000
seen  4000
seen  5000
seen  6000
seen  7000
seen  8000
seen  9000
seen  10000
seen  11000
seen  12000
seen  13000
seen  14000
seen  15000
seen  16000
seen  17000
seen  18000
seen  19000
seen  20000
seen  21000
seen  22000
seen  23000
seen  24000
seen  25000
seen  26000
seen  27000
seen  28000
seen  29000
seen  30000
seen  31000
seen  32000
seen  33000
trying  dcebdcf65a19f9d7
seen  34000
seen  35000
seen  36000


Dataset({
    features: ['topic', 'tags', 'text', 'int_bias', 'summary_5', 'summary_50', 'summary_100', 'text_entity_sentiments', 'text_topic_to_sentiment', 'summary_100_entity_sentiments', 'summary_100_topic_to_sentiment', 'summary_50_entity_sentiments', 'summary_50_topic_to_sentiment', 'id', 'text_encoded', 'summary_100_encoded', 'summary_50_encoded', 'text_topic_0', 'text_sentiment_0', 'text_topic_1', 'text_sentiment_1', 'text_topic_2', 'text_sentiment_2', 'text_topic_3', 'text_sentiment_3', 'text_topic_4', 'text_sentiment_4', 'summary_100_topic_0', 'summary_100_sentiment_0', 'summary_100_topic_1', 'summary_100_sentiment_1', 'summary_100_topic_2', 'summary_100_sentiment_2', 'summary_100_topic_3', 'summary_100_sentiment_3', 'summary_100_topic_4', 'summary_100_sentiment_4', 'summary_50_topic_0', 'summary_50_sentiment_0', 'summary_50_topic_1', 'summary_50_sentiment_1', 'summary_50_topic_2', 'summary_50_sentiment_2', 'summary_50_topic_3', 'summary_50_sentiment_3', 'summary_50_topic_4', 

In [64]:
seen = pd.read_csv("data/summary_5/summary_5_sentiment_llama.csv", sep='|')["id"]
seen = list(set(seen))
get_entity_sentiments_for_text_in_dataset("summary_5", ds_test, seen)

Number of rows to process: 9018
seen  1000
seen  2000
trying  ce3ef16f6eb3a77c
seen  3000
seen  4000
trying  52dc4b7429a6fac6
seen  5000
seen  6000
seen  7000
seen  8000
seen  9000


Dataset({
    features: ['topic', 'tags', 'text', 'int_bias', 'summary_5', 'summary_50', 'summary_100', 'text_entity_sentiments', 'text_topic_to_sentiment', 'summary_100_entity_sentiments', 'summary_100_topic_to_sentiment', 'summary_50_entity_sentiments', 'summary_50_topic_to_sentiment', 'id', 'text_encoded', 'summary_100_encoded', 'summary_50_encoded', 'text_topic_0', 'text_sentiment_0', 'text_topic_1', 'text_sentiment_1', 'text_topic_2', 'text_sentiment_2', 'text_topic_3', 'text_sentiment_3', 'text_topic_4', 'text_sentiment_4', 'summary_100_topic_0', 'summary_100_sentiment_0', 'summary_100_topic_1', 'summary_100_sentiment_1', 'summary_100_topic_2', 'summary_100_sentiment_2', 'summary_100_topic_3', 'summary_100_sentiment_3', 'summary_100_topic_4', 'summary_100_sentiment_4', 'summary_50_topic_0', 'summary_50_sentiment_0', 'summary_50_topic_1', 'summary_50_sentiment_1', 'summary_50_topic_2', 'summary_50_sentiment_2', 'summary_50_topic_3', 'summary_50_sentiment_3', 'summary_50_topic_4', 

In [65]:
seen = pd.read_csv("data/summary_5/summary_5_sentiment_llama.csv", sep='|')

In [66]:
seen = seen.drop_duplicates(subset="id")
seen.shape

(45099, 2)

In [67]:
seen.to_csv("data/summary_5/summary_5_sentiment_llama.csv", sep='|', index=False)